In [12]:
import numpy as np
import plotly.plotly as py
import plotly.graph_objs as go
import pandas as pd

## Zadanie 1
Napisz w dowolnym języku zwyczajną (wolną) dyskretną transformatę Fouriera.

In [13]:
def DFT_slow(x):
    x = np.asarray(x, dtype=float)
    N = x.shape[0]
    
    n = np.arange(N)
    k = n.reshape((N, 1))
    M = np.exp(-2j * np.pi * k * n / N)
    return np.dot(M, x)

In [14]:
x = np.random.random(1024)
np.allclose(DFT_slow(x), np.fft.fft(x))

True

## Zadanie 2
Wykorzystaj implementację z zadania 1.
do napisania szybkiej wersji transformaty (używając pomysłu z algorytmu Cooleya-Tukeya).

In [15]:
def FFT(x):
    x = np.asarray(x, dtype=float)
    N = x.shape[0]
    
    if N % 2 > 0:
        raise ValueError("size of x must be a power of 2, but is ", N)
    elif N <= 16:
        return DFT_slow(x)
    else:
        X_even = FFT(x[::2])
        X_odd = FFT(x[1::2])
        factor = np.exp(-2j * np.pi * np.arange(N) / N)
        return np.concatenate([X_even + factor[:N // 2] * X_odd, X_even + factor[N // 2:] * X_odd])

In [16]:
x = np.random.random(1024)
np.allclose(FFT(x), np.fft.fft(x))

True

#### Porównanie szybkości algorytmów
- wersja wolna
- wersja C-T
- wersja biblioteczna

In [17]:
%timeit DFT_slow(x)
%timeit FFT(x)
%timeit np.fft.fft(x)

224 ms ± 13.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
7.52 ms ± 727 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
8.91 µs ± 502 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Wyniki nie są zaskakujące:
$t(\mathsf{DFT\_slow}) > t(\mathsf{FFT}) > t(\mathsf{np.fft})$

## Zadanie 3
Przetestuj implementację z zadania 2. do wykonania analizy szeregu czasowego:
1. Znajdź dane przedstawiające jakiś szereg czasowy.
2. Załaduj je do programu (polecany: Python + Pandas, ale dowolna metoda jest okej, w tym języki R oraz Julia).
3. Zobacz, czy wykonanie analizy Fouriera na tych danych ma sens -- być może trzeba pogrupować je w równe odstępy (zob: funkcja resample w pakiecie Pandas).
4. Narysuj wykres częstotliwości i postaraj się opisać jaka jest główna składowa.

In [18]:
t = np.linspace(0, 0.5, 512)
s = np.sin(40 * 2 * np.pi * t) + 0.5 * np.sin(90 * 2 * np.pi * t) + 0.3 * np.cos(180 * 2 * np.pi * t)
T = t[1] - t[0]
N = s.size
f = np.linspace(0, 1 / T, N)

fft = FFT(s)

data = [go.Scatter( x=f[:N // 2], y=np.abs(fft)[:N // 2] * 1 / N )]
py.iplot(data, filename='xdd')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~kaskadz/0 or inside your plot.ly account where it is named 'xdd'


In [19]:
url = "global_temperature_monthly.csv"
df = pd.read_csv(url, nrows=1024)
s = df['Mean']

fft = FFT(s)
data = [go.Scatter( x=df['Date'], y=np.abs(fft)[1:N // 2])]
py.iplot(data, filename='fft')

## Pytanie otwarte
Czy transformata Fouriera może zostać wykorzystana do przewidywania szeregów czasowych?

FFT może zostać wykorzystana do przewidywania szeregów czasowych.
Pozwala ona <u>_odróżnić dane występujące **cyklicznie** od **trendu**_</u>, który ma miejsce na przestrzeni czasu.

Link do papera, który pokrywa ten temat: [The Application of Fourier Analysis to Forecasting the Inbound Call Time Series of a Call Centre](http://mssanz.org.au/MODSIM03/Media/Articles/Vol%203%20Articles/1281-1286.pdf)